In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.patches
import pandas as pd
from iminuit import Minuit
import scipy
from chebyshev_fitting import Data
from chebyshev_fitting import Accept
import json
from math import ceil
from sklearn.utils import resample
import csv

In [2]:

#Change this to where acceptance is located
df_accept = pd.read_csv('../data/acceptance_mc.csv') 


#Change this to where the ml signal is located
df_data = pd.read_csv('ML_SIGNAL_BDT.csv')

In [3]:
def cal_observables(df_data, verbose = False):
    
    
    # df_data = pd.read_csv('signal.csv')
    #%% Acceptance:

    acceptance = Accept(df_accept)
    data = Data(df_data)
    bins = []
    bin_edges = ([[0.1,0.98], [1.1,2.5], [2.5,4.0], [4.0,6.0], [6.0,8.0], [15.0,17.0], [17.0,19.0], [11.0,12.5], [1.0,6.0], [15.0,17.9]])

    def bin_q2(dataframe,edges, bins):
        '''
        This function bins given data according to the input bin edges.
        '''
        for i in range(len(edges)):
            df1 = dataframe[dataframe['q2']<edges[i][1]]
            df1 = df1[df1['q2']>edges[i][0]]
            bins.append(df1)
                
    bin_q2(df_data, bin_edges, bins)

    poly_l_list = []
    poly_k_list = []
    poly_p_list = []
    poly_q_list = []

    #Uncomment your choice of polynomial



    for i in range(len(bin_edges)):
        b = bin_edges[i]
        poly_l = acceptance.chebyshev("costhetal", b[0], b[1], order = 4, name = "Data", plot_p = 0, plot_poly = 0)[1]
        poly_k = acceptance.chebyshev("costhetak", b[0], b[1], order = 4, name = "Data", plot_p = 0, plot_poly = 0)[1]
        poly_p = acceptance.chebyshev("phi", b[0], b[1], order = 4, name = "Data", plot_p = 0, plot_poly = 0)[1]
        poly_q = acceptance.chebyshevq(b[0], b[1], order = 5, name = "Data", plot_p = 0, plot_poly = 0)[1]
        poly_l_list.append(poly_l)
        poly_k_list.append(poly_k)
        poly_p_list.append(poly_p)
        poly_q_list.append(poly_q)

    #%% Distributions: Theta - Phi

    def ctl_dist(ctl,q2,fl,afb,_bin):
        c2tl = 2*ctl**2 - 1
        dist = 3/8 * (3/2 - 1/2*fl + 1/2*c2tl*(1-3*fl) + 8/3*afb*ctl)  
        
        poly_l = poly_l_list[int(_bin)]
        poly_q = poly_q_list[int(_bin)]

        acceptance_fn_l = poly_l[0] + poly_l[1]*ctl + poly_l[2]*ctl**2 + poly_l[3]*ctl**3 + poly_l[4]*ctl**4
        acceptance_fn_q = poly_q[0] + poly_q[1]*q2 + poly_q[2]*q2**2 + poly_q[3]*q2**3 + poly_q[4]*q2**4 + poly_q[5]*q2**5

        acceptance_fn = acceptance_fn_l*acceptance_fn_q
    
        normalised_dist = dist*acceptance_fn
        return normalised_dist

    def ctk_dist(ctk,q2,fl,_bin):
        stk2 = 1-ctk**2
        ctk2 = ctk**2
        dist = 3/4 * ((1-fl)*stk2 + 2*fl*ctk2)
        
        poly_k = poly_k_list[int(_bin)]
        poly_q = poly_q_list[int(_bin)]
        
        acceptance_fn_k = poly_k[0] + poly_k[1]*ctk + poly_k[2]*ctk**2 + poly_k[3]*ctk**3 + poly_k[4]*ctk**4
        acceptance_fn_q = poly_q[0] + poly_q[1]*q2 + poly_q[2]*q2**2 + poly_q[3]*q2**3 + poly_q[4]*q2**4 + poly_q[5]*q2**5
        
        acceptance_fn = acceptance_fn_k*acceptance_fn_q
        normalised_dist = dist*acceptance_fn
        return normalised_dist
    
    def phi_dist(phi,q2,s3,s9,_bin):
        dist = 1/(2*np.pi) * (1 + s3*np.cos(2*phi) + s9*np.sin(2*phi))
        
        poly_p = poly_p_list[int(_bin)]
        poly_q = poly_q_list[int(_bin)]

        acceptance_fn_p = poly_p[0] + poly_p[1]*phi + poly_p[2]*phi**2 + poly_p[3]*phi**3 + poly_p[4]*phi**4
        acceptance_fn_q = poly_q[0] + poly_q[1]*q2 + poly_q[2]*q2**2 + poly_q[3]*q2**3 + poly_q[4]*q2**4 + poly_q[5]*q2**5

        acceptance_fn = acceptance_fn_p*acceptance_fn_q
        normalised_dist = dist*acceptance_fn
        return normalised_dist

    #%% Derived Quantities:
        
    def Ps(Ss, FLs, P):
        for i in range(len(Ss)):
            P.append(Ss[i]/(np.sqrt(1-FLs[i])))

    def Ps_E(Ss, FLs, Ss_E, FLs_E,P_E):
        for i in range(len(Ss)):
            A = ((Ss_E[i])/(np.sqrt(1-FLs[i])))
            B = (((Ss[i]*FLs_E[i])/2)*(1/(1-FLs[i])**(3/2)))
            P_E.append(np.sqrt(A**2 + B**2))

    #%% Log-likelihood:

    costhetal = data.angle("costhetal")
    costhetak = data.angle("costhetak")
    d_phi = data.angle("phi")
    d_q2 = data.q2()


    def log_likelihood(FL,AFB,S3,S4,S5,S7,S8,S9,_bin):
        bbin = bins[int(_bin)]
        ctl = bbin['costhetal']
        ctk = bbin['costhetak']
        phi = bbin['phi']
        q2 =  bbin["q2"]
        P = dist(ctk,ctl,phi,q2,FL,AFB,S3,S4,S5,S7,S8,S9,_bin)
        return - np.sum(np.log(P))

    def decay_rate_eqn(ctk,ctl,phi,FL,AFB,S3,S4,S5,S7,S8,S9):
        ctk2 = ctk**2    
        ctl2 = ctl**2
        s2k = 2* np.sqrt(1-ctk2)*ctk
        s2l = 2* np.sqrt(1-ctl2)*ctl
        A = 0.75*(1-FL)*(1-ctk2) + FL*(ctk2) + 0.25*(1-FL)*(1-ctk2)*((2*ctl2)-1)
        B = -FL*(ctk2)*(2*ctl2-1) + S3*(1-ctk**2)*(1-ctl2)*np.cos(2*phi)
        C = S4*s2k*s2l*np.cos(phi) + S5*s2k*(np.sqrt(1-ctl**2))*np.cos(phi)
        D = (4/3)*AFB*(1-ctk2)*ctl + S7*s2k*(np.sqrt(1-ctl**2))*np.sin(phi)
        E = S8*s2k*s2l*np.sin(phi) + S9*(1-ctk2)*(1-ctl2)*np.sin(2*phi)
        F = (9/(32*np.pi))*(A+B+C+D+E)
        return F

    def dist(ctk,ctl,phi,q2,FL,AFB,S3,S4,S5,S7,S8,S9,_bin):
        poly_l = poly_l_list[int(_bin)]
        poly_k = poly_k_list[int(_bin)]
        poly_p = poly_p_list[int(_bin)]
        poly_q = poly_q_list[int(_bin)]
        F = decay_rate_eqn(ctk,ctl,phi,FL,AFB,S3,S4,S5,S7,S8,S9)

        acceptance_fn_l = poly_l[0] + poly_l[1]*ctl + poly_l[2]*ctl**2 + poly_l[3]*ctl**3 + poly_l[4]*ctl**4
        acceptance_fn_k = poly_k[0] + poly_k[1]*ctk + poly_k[2]*ctk**2 + poly_k[3]*ctk**3 + poly_k[4]*ctk**4
        acceptance_fn_p = poly_p[0] + poly_p[1]*phi + poly_p[2]*phi**2 + poly_p[3]*phi**3 + poly_p[4]*phi**4
        acceptance_fn_q = poly_q[0] + poly_q[1]*q2 + poly_q[2]*q2**2 + poly_q[3]*q2**3 + poly_q[4]*q2**4 + poly_q[5]*q2**5
        
        acceptance_fn = acceptance_fn_l*acceptance_fn_k*acceptance_fn_p*acceptance_fn_q
        normalised_dist = (F*acceptance_fn)
        
        return normalised_dist


    #%% Initial Values:

    bin0 = [0.296448,-0.097052,0.010876,0.090919,0.252907,-0.020672,-0.002153,-0.000701]
    bin1 = [0.760396,-0.137987,0.002373,-0.025359,0.054524,-0.027117,-0.006877,-0.000786]
    bin2 = [0.796265,-0.017385,-0.010864,-0.151610,-0.193015,-0.019962,-0.006587,-0.000735]
    bin3 = [0.711290,0.122155,-0.024751,-0.224204,-0.337140,-0.013383,-0.005062,-0.000706]
    bin4 = [0.606965,0.239939,-0.039754,-0.259699,-0.403554,-0.008738,-0.003689,-0.000715]
    bin5 = [0.348441,0.401914,-0.173464,-0.294319,-0.318728,-0.001377,0.000323,0.000292]
    bin6 = [0.328081,0.318391,-0.251488,-0.310007,-0.226258,-0.000561,0.000119,0.000169]
    bin7 = [0.435190,0.391390,-0.085975,-0.281589,-0.406803,-0.002194,0.001051,0.000449]
    bin8 = [0.747644,0.004929,-0.012641,-0.142821,-0.176674,-0.019362,-0.006046,-0.000739]
    bin9 = [0.340156,0.367672,-0.204963,-0.300427,-0.280936,-0.001039,0.000240, 0.000242]

    st=[bin0,bin1,bin2,bin3,bin4,bin5,bin6,bin7,bin8,bin9]

    #%% Minimisation:


    bin_number_to_check = [0,1,2,3,4,5,6,7,8,9]
    bin_results_to_check = None

    log_likelihood.errordef = Minuit.LIKELIHOOD
    decimal_places = 3
    starting_point = st

    FLs, FLs_E = [], []
    S3s, S3s_E = [], []
    S4s, S4s_E = [], []
    S5s, S5s_E = [], []
    AFBs, AFBs_E = [], []
    S7s, S7s_E = [], []
    S8s, S8s_E = [], []
    S9s, S9s_E = [], []

    for i in bin_number_to_check:
        if verbose == True:
            print('Fitting for bin',i)
        m = Minuit(log_likelihood, FL=st[i][0], AFB=st[i][1], S3=st[i][2], S4=st[i][3], S5=st[i][4],S7=st[i][5], S8=st[i][6], S9=st[i][7],  _bin = i)
        m.fixed['_bin'] = True 
        m.limits=((-1.0, 1.0),(-1.0, 1.0),(-1.0, 1.0),(-1.0, 1.0),(-1.0, 1.0),(-1.0, 1.0),(-1.0, 1.0),(-1.0, 1.0), None)
        m.migrad() 
        m.hesse() 
    
        
        FLs.append(m.values[0])
        AFBs.append(m.values[1])
        S3s.append(m.values[2])
        S4s.append(m.values[3])
        S5s.append(m.values[4])
        S7s.append(m.values[5])
        S8s.append(m.values[6])
        S9s.append(m.values[7])
        
        FLs_E.append(m.errors[0])
        AFBs_E.append(m.errors[1])
        S3s_E.append(m.errors[2])
        S4s_E.append(m.errors[3])
        S5s_E.append(m.errors[4])
        S7s_E.append(m.errors[5])
        S8s_E.append(m.errors[6])
        S9s_E.append(m.errors[7])
    return FLs,AFBs,S3s,S4s,S5s,S7s,S8s,S9s

In [4]:
def addToFile(file, msg):
    f = open(file, 'a').write(msg+'\n') 

In [5]:
arr = ['Bin'+str(i) for i in range(10)]

# The Cells below are the only cells you really need to look at!

In [50]:
#RUN THIS CELL ONLY ONCE TO CREATE THE INITAL FILES!!!!!



# addToFile('Bootstrap_thread3/FLs.csv',','.join(list(map(str,arr))))
# addToFile('Bootstrap_thread3/AFBs.csv',','.join(list(map(str,arr))))
# addToFile('Bootstrap_thread3/S3s.csv',','.join(list(map(str,arr))))
# addToFile('Bootstrap_thread3/S4s.csv',','.join(list(map(str,arr))))
# addToFile('Bootstrap_thread3/S5s.csv',','.join(list(map(str,arr))))
# addToFile('Bootstrap_thread3/S7s.csv',','.join(list(map(str,arr))))
# addToFile('Bootstrap_thread3/S8s.csv',','.join(list(map(str,arr))))
# addToFile('Bootstrap_thread3/S9s.csv',','.join(list(map(str,arr))))


In [63]:
import multiprocessing

In [6]:

#Choose the number of iterations you want and run the
n_iterations = 200

for i in range(n_iterations):
    print('SAMPLE',i)
    resampled_data = resample(df_data, replace=True)
    FLs,AFBs,S3s,S4s,S5s,S7s,S8s,S9s = cal_observables(resampled_data, verbose = False)
    addToFile('Bootstrap/FLs.csv',','.join(list(map(str,FLs))))
    addToFile('Bootstrap/AFBs.csv',','.join(list(map(str,AFBs))))
    addToFile('Bootstrap/S3s.csv',','.join(list(map(str,S3s))))
    addToFile('Bootstrap/S4s.csv',','.join(list(map(str,S4s))))
    addToFile('Bootstrap/S5s.csv',','.join(list(map(str,S5s))))
    addToFile('Bootstrap/S7s.csv',','.join(list(map(str,S7s))))
    addToFile('Bootstrap/S8s.csv',','.join(list(map(str,S8s))))
    addToFile('Bootstrap/S9s.csv',','.join(list(map(str,S9s))))


SAMPLE 0
SAMPLE 1
SAMPLE 2
SAMPLE 3
SAMPLE 4
SAMPLE 5
SAMPLE 6
SAMPLE 7
SAMPLE 8
SAMPLE 9
SAMPLE 10
SAMPLE 11
SAMPLE 12
SAMPLE 13
SAMPLE 14
SAMPLE 15
SAMPLE 16
SAMPLE 17
SAMPLE 18


d:\Languages\Python\Python39\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


SAMPLE 19
SAMPLE 20
SAMPLE 21
SAMPLE 22
SAMPLE 23
SAMPLE 24
SAMPLE 25
SAMPLE 26
SAMPLE 27
SAMPLE 28
SAMPLE 29
SAMPLE 30
SAMPLE 31
SAMPLE 32
SAMPLE 33
SAMPLE 34
SAMPLE 35
SAMPLE 36
SAMPLE 37
SAMPLE 38
SAMPLE 39
SAMPLE 40
SAMPLE 41
SAMPLE 42
SAMPLE 43
SAMPLE 44
SAMPLE 45
SAMPLE 46
SAMPLE 47
SAMPLE 48
SAMPLE 49
SAMPLE 50
SAMPLE 51
SAMPLE 52
SAMPLE 53
SAMPLE 54
SAMPLE 55
SAMPLE 56
SAMPLE 57
SAMPLE 58
SAMPLE 59
SAMPLE 60
SAMPLE 61
SAMPLE 62
SAMPLE 63
SAMPLE 64
SAMPLE 65
SAMPLE 66
SAMPLE 67
SAMPLE 68
SAMPLE 69
SAMPLE 70
SAMPLE 71
SAMPLE 72
SAMPLE 73
SAMPLE 74
SAMPLE 75
SAMPLE 76
SAMPLE 77
SAMPLE 78
SAMPLE 79
SAMPLE 80
SAMPLE 81
SAMPLE 82
SAMPLE 83
SAMPLE 84
SAMPLE 85
SAMPLE 86
SAMPLE 87
SAMPLE 88
SAMPLE 89
SAMPLE 90
SAMPLE 91
SAMPLE 92
SAMPLE 93
SAMPLE 94
SAMPLE 95
SAMPLE 96
SAMPLE 97
SAMPLE 98
SAMPLE 99
SAMPLE 100
SAMPLE 101
SAMPLE 102
SAMPLE 103
SAMPLE 104
SAMPLE 105
SAMPLE 106
SAMPLE 107
SAMPLE 108
SAMPLE 109
SAMPLE 110
SAMPLE 111
SAMPLE 112
SAMPLE 113
SAMPLE 114
SAMPLE 115
SAMPLE 116
SAM